In [1]:
import numpy as np
import os
import xml.etree.ElementTree as ET

In [2]:
projectDir = r'C:\Users\Viktor\Skola\embedded\embedded_project'

homeDir = os.path.join(projectDir,'keras_Realtime_Multi-Person_Pose_Estimation-master')
xmlDir = os.path.join(projectDir, 'Stanford40_XMLAnnotations\XMLAnnotations')

#homeDir = r'C:\Users\Viktor\Skola\embedded\embedded_project\keras_Realtime_Multi-Person_Pose_Estimation-master'
#xmlDir = r'C:\Users\Viktor\Skola\embedded\embedded_project\Stanford40_XMLAnnotations\XMLAnnotations'

In [3]:
x = np.load('partCoords.npy')
print(x.shape)

(9532, 18, 2)


In [4]:
os.chdir(homeDir)

jpgOrder = open('JPG_Order.txt', 'r')
lines = jpgOrder.readlines()

y = []

os.chdir(xmlDir)

for i in range(len(lines)):
    imgFileName = lines[i].strip()
    fileName = imgFileName.split(".")[0]
    xmlFileName = fileName + ".xml"
    
    root = ET.parse(xmlFileName).getroot()
    action = root.find('object/action').text
    
    y.append(action)
    
    print(str(i) + '/' + str(len(lines)),end='\r')

print(y)

['applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'applauding', 'appl